In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os

2025-06-03 16:12:59.566744: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-03 16:13:01.282353: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-03 16:13:01.282421: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-03 16:13:01.465201: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-03 16:13:01.855837: I tensorflow/core/platform/cpu_feature_guar

In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
def create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps):
    # Input layer for the encoder
    inputs = layers.Input(shape=(input_timesteps, input_features))

    # Encoder LSTM
    encoder = layers.LSTM(64, activation='relu', return_state=True, return_sequences=False)
    encoder_outputs, state_h, state_c = encoder(inputs)

    # Decoder LSTM: We now provide the encoder's state and initialize it with the encoder's final states
    # We reshape the output of the encoder to make sure it is in the expected form for the decoder
    decoder_input = layers.RepeatVector(output_timesteps)(encoder_outputs)

    # Decoder LSTM, where the output sequence length is `output_timesteps` (7)
    decoder_lstm = layers.LSTM(64, activation='relu', return_sequences=True)
    decoder_outputs = decoder_lstm(decoder_input, initial_state=[state_h, state_c])

    # Dense layer to predict the next 7 time periods (each with 32 features)
    outputs = layers.Dense(input_features)(decoder_outputs)

    # Create the model
    model = models.Model(inputs, outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    
    return model

# Define the input shape and output shape
input_timesteps = 12  # 12 time periods
input_features = 32   # 32 features per time period
output_timesteps = 7  # Predict the next 7 time periods

# Create the model
model = create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps)

# Summarize the model
model.summary()


2025-06-03 16:14:13.411466: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1929] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31134 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:06:00.0, compute capability: 7.0


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 12, 32)]             0         []                            
                                                                                                  
 lstm (LSTM)                 [(None, 64),                 24832     ['input_1[0][0]']             
                              (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                                  
 repeat_vector (RepeatVecto  (None, 7, 64)                0         ['lstm[0][0]']                
 r)                                                                                           

In [3]:
# Example training data (input and target sequences)
import numpy as np

# Random input data (batch_size, timesteps, features)
X_train = np.random.random((10000, input_timesteps, input_features))

# Random target data (batch_size, timesteps, features) - predicted next 7 time periods
y_train = np.random.random((10000, output_timesteps, input_features))

# Train the model
model.fit(X_train, y_train, epochs=100, batch_size=32)


Epoch 1/100


2025-06-03 16:14:21.937476: I external/local_xla/xla/service/service.cc:168] XLA service 0x1503c8c8c860 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-06-03 16:14:21.937529: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Tesla V100S-PCIE-32GB, Compute Capability 7.0
2025-06-03 16:14:22.150741: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-06-03 16:14:22.826522: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8907
I0000 00:00:1748985263.235102 1707106 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


313/313 [==============================] - 15s 28ms/step - loss: 0.0921
Epoch 2/100
313/313 [==============================] - 9s 28ms/step - loss: 0.0837
Epoch 3/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0836
Epoch 4/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0835
Epoch 5/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0835
Epoch 6/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0835
Epoch 7/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0835
Epoch 8/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0835
Epoch 9/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0834
Epoch 10/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0834
Epoch 11/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0834
Epoch 12/100
313/313 [==============================] - 9s 29ms/step - loss: 0.0834
Epoch 13/100

In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
def create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps):
    # Input layer for the encoder
    inputs = layers.Input(shape=(input_timesteps, input_features))

    # Encoder LSTM: Outputs hidden states for each time step in the input sequence
    encoder = layers.LSTM(64, activation='relu', return_state=True, return_sequences=False)
    encoder_outputs, state_h, state_c = encoder(inputs)

    # Decoder LSTM: We will generate output_timesteps (7) future time periods
    decoder_input = layers.RepeatVector(output_timesteps)(encoder_outputs)

    # Decoder LSTM generates the sequence of 7 time periods
    decoder_lstm = layers.LSTM(64, activation='relu', return_sequences=True)
    decoder_outputs = decoder_lstm(decoder_input, initial_state=[state_h, state_c])

    # TimeDistributed layer to predict 32 features for each of the 7 time periods
    time_distributed = layers.TimeDistributed(layers.Dense(input_features))
    outputs = time_distributed(decoder_outputs)

    # Create the model
    model = models.Model(inputs, outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    
    return model

# Define the input shape and output shape
input_timesteps = 12  # 12 time periods (input sequence)
input_features = 32   # 32 features per time period
output_timesteps = 7  # Predict the next 7 time periods

# Create the model
model = create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps)

# Summarize the model
model.summary()


Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 12, 32)]             0         []                            
                                                                                                  
 lstm_2 (LSTM)               [(None, 64),                 24832     ['input_2[0][0]']             
                              (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                                  
 repeat_vector_1 (RepeatVec  (None, 7, 64)                0         ['lstm_2[0][0]']              
 tor)                                                                                       

In [5]:
import tensorflow as tf
from tensorflow.keras import layers, models

# Define the model
def create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps):
    # Input layer for the encoder
    inputs = layers.Input(shape=(input_timesteps, input_features))

    # Encoder LSTM: Outputs the final hidden state (return_sequences=False)
    encoder = layers.LSTM(64, activation='relu', return_state=True, return_sequences=False)
    encoder_outputs, state_h, state_c = encoder(inputs)

    # RepeatVector: Repeat the encoder's final hidden state for `output_timesteps` (7)
    decoder_input = layers.RepeatVector(output_timesteps)(state_h)

    # Decoder LSTM: Uses the repeated encoder final state and generates output_timesteps future time periods
    decoder_lstm = layers.LSTM(64, activation='relu', return_sequences=True)
    decoder_outputs = decoder_lstm(decoder_input, initial_state=[state_h, state_c])

    # TimeDistributed layer to predict 32 features for each of the 7 time periods
    time_distributed = layers.TimeDistributed(layers.Dense(input_features))
    outputs = time_distributed(decoder_outputs)

    # Create the model
    model = models.Model(inputs, outputs)
    
    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    
    return model

# Define the input shape and output shape
input_timesteps = 12  # 12 time periods (input sequence)
input_features = 32   # 32 features per time period
output_timesteps = 7  # Predict the next 7 time periods

# Create the model
model = create_sequence_to_sequence_model(input_timesteps, input_features, output_timesteps)

# Summarize the model
model.summary()


Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_3 (InputLayer)        [(None, 12, 32)]             0         []                            
                                                                                                  
 lstm_4 (LSTM)               [(None, 64),                 24832     ['input_3[0][0]']             
                              (None, 64),                                                         
                              (None, 64)]                                                         
                                                                                                  
 repeat_vector_2 (RepeatVec  (None, 7, 64)                0         ['lstm_4[0][1]']              
 tor)                                                                                       

I think all the three methods are producing the same code, and I think this is correct. We may still need to see if this is correct, or if this needs to be redefned. 

Redefine the code later with correct tf specifications, and I think it should be good.

Also, note that we are using an input size of 12, 32, in reality we have 13, 32 as the input. Adjust for this as well.